In [1]:
# basic imports
import pandas as pd
import sys
import numpy as np
import pandas as pd
import scanpy as sc

# add `Tangram` to path
import tangram as tg

In [2]:
package_version = tg.__version__
print(package_version)

1.0.4


In [5]:
## setting for STARmap
sc_file_path = "/home/comp/cszrwang/project/Geneformer_test/STARmap/toy/starmap_sc_rna.h5ad"
spatial_file_path = "/home/comp/cszrwang/project/Geneformer_test/STARmap/toy/starmap_spatial.h5ad"
celltype_key = 'celltype'
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/STARmap/Tangram/"

In [6]:
ad_sc = sc.read_h5ad(sc_file_path)
ad_sp = sc.read_h5ad(spatial_file_path)

# use raw count both of scrna and spatial
sc.pp.normalize_total(ad_sc)
celltype_counts = ad_sc.obs[celltype_key].value_counts()
celltype_drop = celltype_counts.index[celltype_counts < 2]
print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
ad_sc = ad_sc[~ad_sc.obs[celltype_key].isin(celltype_drop),].copy()
sc.tl.rank_genes_groups(ad_sc, groupby=celltype_key, use_raw=False)
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
print(markers_df)
genes_sc = np.unique(markers_df.melt().value.values)
print(genes_sc)
genes_st = ad_sp.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

Drop celltype [] contain less 2 sample
        Astro     Endo Excitatory L2/3   Excitatory L4 Excitatory L5  \
0      Malat1      Bsg          Ppp3ca           Nrxn1         Mgst3   
1      Slc1a3    Cldn5            Chn1            Rgs4          Chn1   
2       Ntsr2     Ly6e          Fkbp1a           Stx1a          Pcp4   
3       Aldoc    Ly6c1          Cacng3           Mef2c         Stmn1   
4      Slc1a2     Pltp           Ctxn1          Ppp3ca          Pak1   
..        ...      ...             ...             ...           ...   
195       Ubc  Tsc22d3           Mat2b           Exph5           Gda   
196    Lrrc58     F11r          Spred1           Cadm3        Atp5f1   
197   Hsd11b1   Tm4sf1           Sytl2           Ttyh1         Tekt5   
198       ND6     Klf4        Il1rapl1  X9130024F11Rik       Zdhhc23   
199  Tmem176a  Slc38a3           Cadm1            Gas7         Dgat2   

    Excitatory L6 Inhibitory Other Inhibitory Pvalb Inhibitory Sst  \
0           Ctxn1         

In [7]:
tg.pp_adatas(ad_sc, ad_sp, genes=genes)

ad_map = tg.map_cells_to_space(
                   ad_sc,
                   ad_sp,
                   mode='clusters',
                   cluster_label=celltype_key)

tg.project_cell_annotations(ad_map, ad_sp, annotation=celltype_key)

celltype_density = ad_sp.obsm['tangram_ct_pred']
celltype_density = (celltype_density.T/celltype_density.sum(axis=1)).T

celltype_density.to_csv(output_file_path + '/Tangram_result.csv')

INFO:root:459 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:882 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 459 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.613, KL reg: 0.173
Score: 0.797, KL reg: 0.001
Score: 0.797, KL reg: 0.001
Score: 0.797, KL reg: 0.001
Score: 0.797, KL reg: 0.001
Score: 0.798, KL reg: 0.001
Score: 0.798, KL reg: 0.001
Score: 0.798, KL reg: 0.001
Score: 0.798, KL reg: 0.001
Score: 0.798, KL reg: 0.001


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
